In [1]:
import pandas as pd
import bs4
import cloudscraper
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver import ActionChains
import time
from selenium.webdriver.support import expected_conditions as EC

In [2]:
#Substituir o arquivo para um csv com uma coluna com os tickers dos fundos imobiliários
df = pd.read_csv('fiis.csv', encoding='utf-8', sep=';')
scrapper = cloudscraper.create_scraper()
tickers = df['Tickers'].tolist()

In [3]:
def get_fii_info(ticker, driver):
    html = ""
    CRIs = 0
    driver.get(f'https://statusinvest.com.br/fundos-imobiliarios/{ticker}')
    try:    
        
        try: 
            advertasing(driver)
            element = driver.find_element(By.XPATH, """//*[@id="main-2"]/div[1]/div/div/ul/li[3]/a""" )
            element.click()
            
            advertasing(driver)
            button = driver.find_elements(By.XPATH, """//*[@title="Ver tudo"]""")[0]
            driver.execute_script("arguments[0].click();", button)
            
            advertasing(driver)
            div = driver.find_element(By.XPATH, """//*[@data-dre-grid]""")
            ActionChains(driver).move_to_element(div).perform()
            time.sleep(5)
            advertasing(driver)
            html = div.get_attribute('outerHTML')
        except Exception as e:
            print("Erro ao pegar o html")
        
        try:
            advertasing(driver)
            total = 0
            data_next = driver.find_element(By.XPATH, """//div[@id="CRI-f-assets"]//li[@data-next="1"]""")
            while data_next.get_attribute("class") != "disabled":
                driver.execute_script("arguments[0].click();", data_next)
                total += 10
                data_next = driver.find_element(By.XPATH, """//div[@id="CRI-f-assets"]//li[@data-next="1"]""")
                advertasing(driver)
      
            advertasing(driver)
            l = driver.find_elements(By.XPATH, """//td[@data-key="Serie"]""")
            CRIs = total + len(l)
            
        except Exception as e:
            print("Erro ao contar CRIs")
        
    except Exception as e:
        print("Erro geral")

    total_investido = ""
    certificados = ""
    if html != "":
        soup = bs4.BeautifulSoup(html, 'html.parser')
        tds = soup.find_all('td')
        for td in tds:
            if td.text == 'Total investido - (R$)':
                total_investido = td.next_sibling.text
                
            if td.text == 'Certificado de Recebíveis Imobiliários (CRI) - (R$)':
                certificados = td.next_sibling.text
                
    return total_investido, certificados, CRIs

In [4]:
tick = "HGRE11"
for i, ti in enumerate(tickers):
    if tick == ti:
        print(i, ti)
        break


270 HGRE11


In [5]:
def advertasing(driver):
    try:
        driver.find_element(By.XPATH, """//button[@class="btn-close"]""").click()
    except:
        pass

In [6]:
dict_fiis = {}
df_fiis = pd.DataFrame(columns=['Ticker', 'Total Investido', 'Certificados', 'CRIs'])

In [42]:
len(tickers)

375

In [7]:
tickers10 = tickers[270:]

In [8]:
driver = webdriver.Chrome()
try:
    advertising = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CLASS_NAME, "a_d__v_e__r_t__i_s__i_n__g.mb-1.undefined.d-block"))
    )
    print("Propaganda detectada")
    button_close = driver.find_element(By.CLASS_NAME, "btn-close")
    driver.execute_script("arguments[0].click();", button_close)
    print("Propaganda fechada com sucesso")
except Exception as e:
        print("Erro propaganda")
for ticker in tickers10:
    print(f"Getting {ticker} info")
    total_investido, certificados, CRIs = get_fii_info(ticker, driver)
    print(f"Total investido: {total_investido}")
    print(f"Certificados: {certificados}")
    print(f"CRIs: {CRIs}")
    dict_fiis[ticker] = {
        'Total investido': total_investido,
        'Certificados': certificados,
        'CRIs': CRIs
    }
    print("")
    
driver.quit()

The chromedriver version (127.0.6533.119) detected in PATH at chromedriver.EXE might not be compatible with the detected chrome version (128.0.6613.85); currently, chromedriver 128.0.6613.86 is recommended for chrome 128.*, so it is advised to delete the driver in PATH and retry


Erro propaganda
Getting HGRE11 info
Total investido: 1.802,97 M
Certificados: 55,66 M
CRIs: 3

Getting HGFF11 info
Total investido: 246,68 M
Certificados: 3,71 M
CRIs: 2

Getting HGLG11 info
Total investido: 5.856,08 M
Certificados: 248,98 M
CRIs: 15

Getting PATL11 info
Erro ao contar CRIs
Total investido: 485,81 M
Certificados: 0,00
CRIs: 0

Getting HGPO11 info
Erro ao contar CRIs
Total investido: 603,05 M
Certificados: 0,00
CRIs: 0

Getting HGCR11 info
Total investido: 1.636,14 M
Certificados: 1.385,33 M
CRIs: 44

Getting HGRU11 info
Total investido: 2.409,37 M
Certificados: 82,74 M
CRIs: 5

Getting PEMA11 info
Total investido: 37,33 M
Certificados: 37,33 M
CRIs: 22

Getting PRSN11 info
Erro ao pegar o html
Erro ao contar CRIs
Total investido: 
Certificados: 
CRIs: 0

Getting PLCA11 info
Erro ao pegar o html
Erro ao contar CRIs
Total investido: 
Certificados: 
CRIs: 0

Getting PLCR11 info
Total investido: 168,75 M
Certificados: 168,75 M
CRIs: 46

Getting PORD11 info
Total investido:

In [57]:
dict_fiis

{'APXM11': {'Total investido': '163,47 M', 'Certificados': '0,00', 'CRIs': 0},
 'FZDB11': {'Total investido': '', 'Certificados': '', 'CRIs': 0},
 'AFHI11': {'Total investido': '403,29 M',
  'Certificados': '403,29 M',
  'CRIs': 52},
 'AJFI11': {'Total investido': '393,09 M',
  'Certificados': '3,45 M',
  'CRIs': 1},
 'ALZC11': {'Total investido': '', 'Certificados': '', 'CRIs': 0},
 'ALZM11': {'Total investido': '119,96 M',
  'Certificados': '56,20 M',
  'CRIs': 22},
 'ALZR11': {'Total investido': '1.318,17 M',
  'Certificados': '20,46 M',
  'CRIs': 8},
 'AURB11': {'Total investido': '155,67 M', 'Certificados': '0,00', 'CRIs': 0},
 'ANCR11': {'Total investido': '', 'Certificados': '', 'CRIs': 0},
 'FAED11': {'Total investido': '149,27 M', 'Certificados': '0,00', 'CRIs': 0},
 'AROA11': {'Total investido': '163,67 M', 'Certificados': '0,00', 'CRIs': 0},
 'ARXD11': {'Total investido': '64,66 M',
  'Certificados': '64,66 M',
  'CRIs': 15},
 'ASMT11': {'Total investido': '149,33 M', 'Certi

In [9]:
df_fiis = pd.DataFrame.from_dict(dict_fiis, orient='index')

In [10]:
df_fiis_anterior = pd.read_csv('fiis_info.csv', encoding='utf-8', sep=';')

In [19]:
df_fiis_anterior.set_index('Ticker', inplace=True)

In [21]:
df_final = pd.concat([df_fiis, df_fiis_anterior], axis=0)

In [42]:
df_final["Total investido"] = df_final["Total investido"].str.replace("M", "")
df_final["Certificados"] = df_final["Certificados"].str.replace("M", "")

In [51]:
df_final["Total investido"] = df_final["Total investido"].apply(lambda x: 0 if pd.isnull(x) else x)
df_final["Certificados"] = df_final["Certificados"].apply(lambda x: 0 if pd.isnull(x) else x)
df_final["CRIs"] = df_final["CRIs"].apply(lambda x: 0 if pd.isnull(x) else x)

df_final["Total investido"] = df_final["Total investido"].replace(" ", "")
df_final["Certificados"] = df_final["Certificados"].replace(" ", "")

In [52]:
df_final.head(30)

,Total investido,Certificados,CRIs
HGRE11,"1.802,97","55,66",3
HGFF11,"246,68","3,71",2
HGLG11,"5.856,08","248,98",15
PATL11,"485,81","0,00",0
HGPO11,"603,05","0,00",0
HGCR11,"1.636,14","1.385,33",44
HGRU11,"2.409,37","82,74",5
PEMA11,"37,33","37,33",22
PRSN11,0,0,0
PLCA11,0,0,0


In [53]:
df_final.to_csv('final3.csv', sep=';', encoding='utf-8')